# Фінальний проєкт

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, TargetEncoder
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from scipy.stats import randint

### Завантажуємо дані

In [3]:
train_data = pd.read_csv('../datasets/final_proj_data.csv')
test_data = pd.read_csv('../datasets/final_proj_test.csv')
train_data.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,Var11,Var12,Var13,Var14,Var15,Var16,Var17,Var18,Var19,Var20,Var21,Var22,Var23,Var24,Var25,Var26,Var27,Var28,Var29,Var30,Var31,Var32,Var33,Var34,Var35,Var36,Var37,Var38,Var39,Var40,...,Var192,Var193,Var194,Var195,Var196,Var197,Var198,Var199,Var200,Var201,Var202,Var203,Var204,Var205,Var206,Var207,Var208,Var209,Var210,Var211,Var212,Var213,Var214,Var215,Var216,Var217,Var218,Var219,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230,y
0,NaN,NaN,NaN,NaN,NaN,812.0,14.0,NaN,NaN,NaN,NaN,NaN,1252.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,156.0,195.0,NaN,0.0,40.0,NaN,NaN,286.96,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,4850466.0,NaN,NaN,...,KXMrEyXXnK,g62hiBSaKg,SEuy,taul,1K8T,IvdZ,fhk21Ss,Pkku4gO,pbHkKHa,smXZ,4MV4,9_Y1,SkZj,VpdQ,itlM,7M47J5GA0pTYIFxg5uy,kIsH,NaN,uKAI,L84s,WsRVNrF85oPU_,NaN,E4QF4eX,NaN,11pRpOD,G8WR,cJvF,AU8_WTd,4UxGlow,zCkv,catzS2D,jySVZNlOJy,NaN,xG3x,Aoh3,ZI9m,ib5G6X1eUxUn6,mj86,NaN,0
1,NaN,NaN,NaN,NaN,NaN,2688.0,7.0,NaN,NaN,NaN,NaN,NaN,8820.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,364.0,455.0,NaN,4.0,288.0,NaN,NaN,200.00,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,132072.0,NaN,NaN,...,8Knvyx875g,2Knk1KF,SEuy,taul,1K8T,lK27,eQgnKxT,Hz673939hbJdw,ELe4G18,smXZ,sbhw,9_Y1,MBhA,VpdQ,lVqb,me75fM6ugJ,kIsH,NaN,uKAI,L84s,XfqtO3UdzaXh_,NaN,kfKvbJ9,NaN,11pEY2j,5smi,UYBR,AU8pNoi,GpvRJ5l,oslk,i06ocsg,LM8l689qOp,NaN,kG3k,WqMG,RAYp,55YFVY9,mj86,NaN,0
2,NaN,NaN,NaN,NaN,NaN,1015.0,14.0,NaN,NaN,NaN,NaN,NaN,1784.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,136.0,170.0,NaN,2.0,40.0,NaN,NaN,294.48,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,3223524.0,NaN,NaN,...,MfKrEyQtC3,RO12,SEuy,taul,1K8T,487l,iJzviRg,76j2P_OLn0,pOtdZA9,smXZ,7KBn,9_Y1,SkZj,VpdQ,hAFG,7M47J5GA0pTYIFxg5uy,kIsH,NaN,g5HH,L84s,cOJGGr4F_wga1,NaN,3_oHWGU,NaN,kq0mb4H,8m7I,cJvF,FzaX,ch2oGfM,zCkv,P6pu4Vl,LM8l689qOp,NaN,kG3k,Aoh3,ZI9m,R4y5gQQWY8OodqDV,am7c,NaN,0
3,NaN,NaN,NaN,NaN,NaN,168.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0,30.0,NaN,0.0,0.0,NaN,NaN,644.24,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,2135430.0,NaN,NaN,...,Qu0qrQKzJV,RO12,NaN,taul,1K8T,487l,3cunsla,ZIX6Y9cVy5,NaN,NaN,6lcF,9_Y1,15m3,09_Q,zm5i,me75fM6ugJ,kIsH,NaN,uKAI,Mtgm,NhsEn4L,NaN,NaN,NaN,7Wwz9n7,d4ij,cJvF,FzaX,kH5mFX7,oslk,BNrD3Yd,LM8l689qOp,NaN,NaN,FSa2,RAYp,F2FyR07IdsN7I,NaN,NaN,0
4,NaN,NaN,NaN,NaN,NaN,14.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.0,45.0,NaN,0.0,0.0,NaN,NaN,239.84,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,3110400.0,NaN,NaN,...,DHerjIKQ76,RO12,NaN,taul,1K8T,YPhp,65ylkJq,glRbITbnHQ,NaN,NaN,UCEj,9_Y1,7WNq,VpdQ,IYzP,me75fM6ugJ,kIsH,NaN,uKAI,L84s,NhsEn4L,NaN,NaN,NaN,kZJtVhC,NEOV,cJvF,FzaX,x_lYlW4,oslk,3B1QowC,LM8l689qOp,NaN,NaN,WqMG,RAYp,F2FyR07IdsN7I,NaN,NaN,0


### Аналізуємо дані

In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 231 entries, Var1 to y
dtypes: float64(191), int64(2), object(38)
memory usage: 17.6+ MB


In [5]:
train_data.describe()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,Var11,Var12,Var13,Var14,Var15,Var16,Var17,Var18,Var19,Var20,Var21,Var22,Var23,Var24,Var25,Var26,Var27,Var28,Var29,Var30,Var31,Var32,Var33,Var34,Var35,Var36,Var37,Var38,Var39,Var40,...,Var154,Var155,Var156,Var157,Var158,Var159,Var160,Var161,Var162,Var163,Var164,Var165,Var166,Var167,Var168,Var169,Var170,Var171,Var172,Var173,Var174,Var175,Var176,Var177,Var178,Var179,Var180,Var181,Var182,Var183,Var184,Var185,Var186,Var187,Var188,Var189,Var190,Var209,Var230,y
count,133.000000,266.0,266.000000,280.000000,2.410000e+02,8980.000000,8995.000000,0.0,133.000000,2.410000e+02,266.000000,104.000000,8995.000000,266.000000,0.0,241.000000,280.000000,280.000000,280.000000,0.0,8980.000000,9080.000000,241.000000,8640.000000,9080.000000,241.000000,241.000000,9080.000000,133.000000,133.000000,0.0,0.0,1.350000e+02,266.000000,9080.000000,2.660000e+02,2.800000e+02,9.080000e+03,0.0,266.000000,...,1.330000e+02,280.000000,131.000000,171.000000,145.000000,266.000000,9080.000000,280.000000,2.660000e+02,9.080000e+03,280.000000,1.450000e+02,241.000000,0.0,133.000000,0.0,266.000000,1.700000e+02,241.000000,9080.000000,280.000000,0.0,266.000000,2.660000e+02,131.000000,280.000000,1.330000e+02,9080.000000,2.800000e+02,2.660000e+02,266.000000,0.0,133.000000,133.000000,266.000000,4206.000000,43.000000,0.0,0.0,10000.00000
mean,14.977444,0.0,341.052632,0.096429,2.338101e+05,1340.916258,6.860700,NaN,61.383459,3.672943e+05,8.421053,23.692308,1312.861812,0.721805,NaN,114.262573,11.589286,6.321429,0.321429,NaN,240.702895,297.810573,8.153527,4.558333,98.824670,0.049793,0.016598,225.037534,0.015038,7.443609,NaN,NaN,9.126613e+04,0.781955,0.740639,1.459700e+05,6.038755e+05,2.591752e+06,NaN,9.774436,...,1.607406e+06,1.303571,217.801527,37.099415,1.841379,4.466165,39.204405,3.085714,3.153896e+05,4.839801e+05,1.607143,3.182265e+04,25.589212,NaN,329.273985,NaN,2.109023,3.362311e+05,9.875519,0.004846,9.614286,NaN,7.458647,6.688716e+05,12.404580,3.000000,3.727323e+06,0.586674,1.333466e+06,6.326152e+04,6.180451,NaN,2.977444,20.601504,159.107368,272.455064,25725.112326,NaN,NaN,0.13050
std,66.456008,0.0,2810.606975,0.928243,5.532305e+05,2380.516758,6.300994,NaN,266.124849,8.234215e+05,2.156904,117.241687,2814.738176,3.647389,NaN,76.529406,56.981031,30.946773,2.131074,NaN,553.289430,688.436748,55.123094,9.232289,215.059736,0.471710,0.181815,102.007658,0.173422,10.011670,NaN,NaN,2.041744e+05,3.164278,3.075834,3.082483e+05,1.247984e+06,3.027801e+06,NaN,28.119051,...,2.278667e+06,4.358157,938.259005,172.819956,5.625680,11.417662,92.790289,8.011627,9.904012e+05,8.499511e+05,7.786761,1.135068e+05,123.618572,NaN,92.672758,NaN,4.224742,4.877593e+05,9.130595,0.098332,70.030161,NaN,57.446088,1.432175e+06,16.702731,21.995438,3.709497e+06,2.470885,2.227988e+06,1.623765e+05,12.177204,NaN,10.329764,93.736247,115.766972,86.752531,37487.484852,NaN,NaN,0.33687
min,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,NaN,0.000000,0.000000e+00,8.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,NaN,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,NaN,0.000000,NaN,0.000000,0.000000e+00,0.000000,0.000000,0.000000,NaN,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,NaN,0.000000,0.000000,0.000000,6.000000,0.000000,NaN,NaN,0.00000
25%,0.000000,0.0,0.000000,0.000000,0.000000e+00,523.250000,0.000000,NaN,2.000000,0.000000e+00,8.000000,0.000000,0.000000,0.000000,NaN,49.080000,0.000000,0.000000,0.000000,NaN,112.000000,135.000000,0.000000,0.000000,16.000000,0.000000,0.000000,166.560000,0.000000,5.000000,NaN,NaN,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,8.553000e+03,NaN,0.000000,...,0.000000e+00,0.000000,

### Перевіряємо пропуски та видаляємо колонки з пропусками >= 50%

In [8]:
train_data.isna().mean().sort_values(ascending=False)
test_data.isna().mean().sort_values(ascending=False)

train_data = train_data[train_data.columns[train_data.isna().mean().lt(0.5)]]
test_data = test_data[test_data.columns[test_data.isna().mean().lt(0.5)]]

### Створюємо числовий та категоріальний трансформери та препроцесор

In [9]:
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer()),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', TargetEncoder(random_state=42))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, make_column_selector(dtype_include=np.number)),
        ('cat', categorical_transformer, make_column_selector(dtype_include=object))
    ]
)

### Перевіряємо розподіл цільової змінної y

In [10]:
train_data['y'].value_counts()

y
0    8695
1    1305
Name: count, dtype: int64

### Будуємо конвеєр з GradientBoostingClassifier моделлю

In [11]:
smote = SMOTE(random_state=42)

model = GradientBoostingClassifier(random_state=42)

clf = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('smote', smote),
    ('classifier', model)
])

### Поділяємо дані на навчальну та тестову вибірки

In [12]:
X = train_data.drop('y', axis=1)
y = train_data['y']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

### Знайдемо найкращі гіперпараметри моделі з використанням RandomizedSearchCV

In [19]:
params = {
    'classifier__n_estimators': randint(100, 200),
    'classifier__learning_rate': [0.01, 0.1],
    'classifier__max_depth': randint(3, 6),
    'classifier__min_samples_split': randint(2, 6),
    'classifier__min_samples_leaf': randint(1, 3)
}

random_search = RandomizedSearchCV(clf, params, n_iter=5, cv=3, scoring='f1', n_jobs=-1, random_state=42)

random_search.fit(X_train, y_train)

random_search.best_params_

{'classifier__learning_rate': 0.1,
 'classifier__max_depth': 4,
 'classifier__min_samples_leaf': 2,
 'classifier__min_samples_split': 3,
 'classifier__n_estimators': 163}

### Отримаємо модель з оптимальними гіперпараметрами

In [20]:
opt_model = random_search.best_estimator_

### Оцінюємо модель на валідаційних даних

In [22]:
y_valid_pred = opt_model.predict(X_valid)
print(classification_report(y_valid, y_valid_pred))

              precision    recall  f1-score   support

           0       0.99      0.85      0.92      1737
           1       0.49      0.92      0.64       263

    accuracy                           0.86      2000
   macro avg       0.74      0.89      0.78      2000
weighted avg       0.92      0.86      0.88      2000



### Виконаємо крос-валідацію

In [23]:
cv_scores = cross_val_score(opt_model, X, y, cv=5)
cv_scores.mean()

0.8444

### Тренуємо модель на повному наборі даних

In [24]:
opt_model.fit(X, y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001EDBC0CF610>),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   TargetEncoder(random_state=42))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001EDBC061E10>)])),
                ('smote', SMOTE(random_state=42)),
                ('classifier',
                 GradientBoostingClassifier(max_depth=4, min_samples_leaf=2,
                                            min_samples_split=3,
                                            n_estimators=163,
                                            random_state=42))])

### Прогнозуємо на тестовому наборі даних

In [25]:
y_test_pred = opt_model.predict(test_data)

### Зберігаємо результати в csv

In [26]:
output = pd.DataFrame({'index': test_data.index, 'y': y_test_pred})
output.to_csv('../datasets/final_proj_predictions.csv', index=False)
output.head()

,index,y
0,0,0
1,1,1
2,2,0
3,3,0
4,4,0
